In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
%cd /content/gdrive/Shareddrives/Anomaly/

/content/gdrive/Shareddrives/Anomaly


In [3]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

from torch.utils.data import DataLoader
#from autoencoder import Autoencoder
import matplotlib.pyplot as plt
from torch.utils.data.dataset import Dataset
from torch.autograd import Variable
import torch.autograd as autograd
import argparse
import os
import random
from collections import OrderedDict

import yaml
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import scipy.stats as stats

In [4]:
train_df = pd.read_csv('/content/gdrive/Shareddrives/Anomaly/dataset/TRAIN_FINAL_CROSS_DOM_3K.csv')

In [5]:
train_df.head()

,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,...,Pkt Len Std,Pkt Len Var,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Idle Min,Label
0,-0.632694,-0.580092,-0.881051,-0.598698,-0.686276,-0.635757,-0.698694,-0.588015,-0.085003,0.369437,...,-0.694505,-0.517268,-0.835366,-0.881051,-0.698694,0.130146,-0.314942,0.680731,-0.344192,0.0
1,-0.712442,1.429695,-0.374214,-0.852608,-0.536097,3.273131,0.201001,-0.794297,-0.068330,-0.136598,...,-0.490374,-0.598422,0.082195,-0.374214,0.201001,-0.755270,-0.426041,-1.445423,-0.361189,0.0
2,-0.632694,-0.580092,-0.881051,-0.598698,-0.686276,-0.635757,-0.698694,-0.588015,-0.085003,-0.164555,...,-0.694505,-0.517268,-0.835366,-0.881051,-0.698694,-0.468538,-0.314942,-2.426335,1.906829,0.0
3,-0.512219,0.741109,-0.261913,-0.598698,-0.468995,1.592125,-0.091600,-0.588015,-0.081888,-0.163937,...,-0.416409,-0.490281,-0.084858,-0.261913,-0.091600,-0.468538,-0.314942,-1.183509,-0.344192,0.0
4,1.709488,-0.580092,1.686548,1.800385,1.352691,-0.635757,-0.000646,0.862475,-0.084752,-0.164529,...,0.844407,0.309143,0.273510,1.686548,-0.000646,0.481678,4.010477,0.680731,-0.344192,0.0


In [6]:
train_df.shape

(156903, 30)

In [7]:
train_df['Label'].value_counts()

0.0    102393
1.0     54510
Name: Label, dtype: int64

In [8]:
X_train = train_df.drop(labels = ['Label'], axis=1)

In [9]:
y_train = train_df['Label']

In [10]:
y_train[y_train>1.0] = 1

In [11]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

In [12]:
BOT_test = pd.read_csv('/content/gdrive/Shareddrives/Anomaly/dataset/BOT.csv')

In [13]:
BOT_test.isna().sum()

Fwd Pkt Len Max      0
Fwd Pkt Len Min      0
Fwd Pkt Len Mean     0
Fwd Pkt Len Std      0
Bwd Pkt Len Max      0
Bwd Pkt Len Min      0
Bwd Pkt Len Mean     0
Bwd Pkt Len Std      0
Flow Byts/s          0
Flow Pkts/s          0
Bwd IAT Tot          0
Bwd IAT Mean         0
Bwd IAT Std          0
Bwd IAT Max          0
Bwd IAT Min          0
Fwd Pkts/s           0
Bwd Pkts/s           0
Pkt Len Min          0
Pkt Len Max          0
Pkt Len Mean         0
Pkt Len Std          0
Pkt Len Var          0
Pkt Size Avg         0
Fwd Seg Size Avg     0
Bwd Seg Size Avg     0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Fwd Seg Size Min     0
Idle Min             0
Label                0
dtype: int64

In [14]:
X_BOT_test = BOT_test.drop(labels = ['Label'], axis=1)

In [15]:
X_BOT_test = X_BOT_test.to_numpy()

In [16]:
y_BOT_test = BOT_test['Label']

In [17]:
y_BOT_test.value_counts()

1.0    10000
Name: Label, dtype: int64

In [18]:
y_BOT_test[y_BOT_test>1.0] = 1.0

In [19]:
y_BOT_test = y_BOT_test.to_numpy()

In [20]:
RARE_test = pd.read_csv('/content/gdrive/Shareddrives/Anomaly/dataset/RARE.csv')

In [21]:
RARE_test.isna().sum()

Fwd Pkt Len Max      0
Fwd Pkt Len Min      0
Fwd Pkt Len Mean     0
Fwd Pkt Len Std      0
Bwd Pkt Len Max      0
Bwd Pkt Len Min      0
Bwd Pkt Len Mean     0
Bwd Pkt Len Std      0
Flow Byts/s          0
Flow Pkts/s          0
Bwd IAT Tot          0
Bwd IAT Mean         0
Bwd IAT Std          0
Bwd IAT Max          0
Bwd IAT Min          0
Fwd Pkts/s           0
Bwd Pkts/s           0
Pkt Len Min          0
Pkt Len Max          0
Pkt Len Mean         0
Pkt Len Std          0
Pkt Len Var          0
Pkt Size Avg         0
Fwd Seg Size Avg     0
Bwd Seg Size Avg     0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Fwd Seg Size Min     0
Idle Min             0
Label                0
dtype: int64

In [22]:
X_RARE_test = RARE_test.drop(labels = ['Label'], axis=1)

In [23]:
X_RARE_test = X_RARE_test.to_numpy()

In [24]:
y_RARE_test = RARE_test['Label']

In [25]:
y_RARE_test.value_counts()

1.0    566
Name: Label, dtype: int64

In [26]:
y_RARE_test[y_RARE_test>1.0] = 1.0

In [27]:
y_RARE_test = y_RARE_test.to_numpy()

In [28]:
SlowHTTPTest_test = pd.read_csv('/content/gdrive/Shareddrives/Anomaly/dataset/SlowHTTPTest.csv')

In [29]:
SlowHTTPTest_test.isna().sum()

Fwd Pkt Len Max      0
Fwd Pkt Len Min      0
Fwd Pkt Len Mean     0
Fwd Pkt Len Std      0
Bwd Pkt Len Max      0
Bwd Pkt Len Min      0
Bwd Pkt Len Mean     0
Bwd Pkt Len Std      0
Flow Byts/s          0
Flow Pkts/s          0
Bwd IAT Tot          0
Bwd IAT Mean         0
Bwd IAT Std          0
Bwd IAT Max          0
Bwd IAT Min          0
Fwd Pkts/s           0
Bwd Pkts/s           0
Pkt Len Min          0
Pkt Len Max          0
Pkt Len Mean         0
Pkt Len Std          0
Pkt Len Var          0
Pkt Size Avg         0
Fwd Seg Size Avg     0
Bwd Seg Size Avg     0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Fwd Seg Size Min     0
Idle Min             0
Label                0
dtype: int64

In [30]:
X_SlowHTTPTest_test = SlowHTTPTest_test.drop(labels = ['Label'], axis=1)

In [31]:
X_SlowHTTPTest_test = X_SlowHTTPTest_test.to_numpy()

In [32]:
y_SlowHTTPTest_test = SlowHTTPTest_test['Label']

In [33]:
y_SlowHTTPTest_test.value_counts()

1.0    461912
Name: Label, dtype: int64

In [34]:
y_SlowHTTPTest_test[y_SlowHTTPTest_test>1.0] = 1.0

In [35]:
y_SlowHTTPTest_test = y_SlowHTTPTest_test.to_numpy()

In [36]:
OOD_INFIL_test = pd.read_csv('/content/gdrive/Shareddrives/Anomaly/dataset/OOD_INFIL.csv')

In [37]:
OOD_INFIL_test.isna().sum()

Fwd Pkt Len Max      0
Fwd Pkt Len Min      0
Fwd Pkt Len Mean     0
Fwd Pkt Len Std      0
Bwd Pkt Len Max      0
Bwd Pkt Len Min      0
Bwd Pkt Len Mean     0
Bwd Pkt Len Std      0
Flow Byts/s          0
Flow Pkts/s          0
Bwd IAT Tot          0
Bwd IAT Mean         0
Bwd IAT Std          0
Bwd IAT Max          0
Bwd IAT Min          0
Fwd Pkts/s           0
Bwd Pkts/s           0
Pkt Len Min          0
Pkt Len Max          0
Pkt Len Mean         0
Pkt Len Std          0
Pkt Len Var          0
Pkt Size Avg         0
Fwd Seg Size Avg     0
Bwd Seg Size Avg     0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Fwd Seg Size Min     0
Idle Min             0
Label                0
dtype: int64

In [38]:
X_OOD_INFIL_test = OOD_INFIL_test.drop(labels = ['Label'], axis=1)

In [39]:
X_OOD_INFIL_test = X_OOD_INFIL_test.to_numpy()

In [40]:
y_OOD_INFIL_test = OOD_INFIL_test['Label']

In [41]:
y_OOD_INFIL_test.value_counts()

1.0    49504
Name: Label, dtype: int64

In [42]:
y_OOD_INFIL_test[y_OOD_INFIL_test>1.0] = 1.0

In [43]:
y_OOD_INFIL_test = y_OOD_INFIL_test.to_numpy()

In [44]:
DDOS_SOLARIS_test = pd.read_csv('/content/gdrive/Shareddrives/Anomaly/dataset/DDOS_SOLARIS.csv')

In [45]:
DDOS_SOLARIS_test.isna().sum()

Fwd Pkt Len Max      0
Fwd Pkt Len Min      0
Fwd Pkt Len Mean     0
Fwd Pkt Len Std      0
Bwd Pkt Len Max      0
Bwd Pkt Len Min      0
Bwd Pkt Len Mean     0
Bwd Pkt Len Std      0
Flow Byts/s          0
Flow Pkts/s          0
Bwd IAT Tot          0
Bwd IAT Mean         0
Bwd IAT Std          0
Bwd IAT Max          0
Bwd IAT Min          0
Fwd Pkts/s           0
Bwd Pkts/s           0
Pkt Len Min          0
Pkt Len Max          0
Pkt Len Mean         0
Pkt Len Std          0
Pkt Len Var          0
Pkt Size Avg         0
Fwd Seg Size Avg     0
Bwd Seg Size Avg     0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Fwd Seg Size Min     0
Idle Min             0
Label                0
dtype: int64

In [46]:
X_DDOS_SOLARIS_test = DDOS_SOLARIS_test.drop(labels = ['Label'], axis=1)

In [47]:
X_DDOS_SOLARIS_test = X_DDOS_SOLARIS_test.to_numpy()

In [48]:
y_DDOS_SOLARIS_test = DDOS_SOLARIS_test['Label']

In [49]:
y_DDOS_SOLARIS_test.value_counts()

1.0    10990
Name: Label, dtype: int64

In [50]:
y_DDOS_SOLARIS_test[y_DDOS_SOLARIS_test>1.0] = 1.0

In [51]:
y_DDOS_SOLARIS_test = y_DDOS_SOLARIS_test.to_numpy()

In [52]:
DDOS_HOIC_test = pd.read_csv('/content/gdrive/Shareddrives/Anomaly/dataset/DDOS_HOIC.csv')

In [53]:
DDOS_HOIC_test.isna().sum()

Fwd Pkt Len Max      0
Fwd Pkt Len Min      0
Fwd Pkt Len Mean     0
Fwd Pkt Len Std      0
Bwd Pkt Len Max      0
Bwd Pkt Len Min      0
Bwd Pkt Len Mean     0
Bwd Pkt Len Std      0
Flow Byts/s          0
Flow Pkts/s          0
Bwd IAT Tot          0
Bwd IAT Mean         0
Bwd IAT Std          0
Bwd IAT Max          0
Bwd IAT Min          0
Fwd Pkts/s           0
Bwd Pkts/s           0
Pkt Len Min          0
Pkt Len Max          0
Pkt Len Mean         0
Pkt Len Std          0
Pkt Len Var          0
Pkt Size Avg         0
Fwd Seg Size Avg     0
Bwd Seg Size Avg     0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Fwd Seg Size Min     0
Idle Min             0
Label                0
dtype: int64

In [54]:
X_DDOS_HOIC_test = DDOS_HOIC_test.drop(labels = ['Label'], axis=1)

In [55]:
X_DDOS_HOIC_test = X_DDOS_HOIC_test.to_numpy()

In [56]:
y_DDOS_HOIC_test = DDOS_HOIC_test['Label']

In [57]:
y_DDOS_HOIC_test.value_counts()

1.0    686012
Name: Label, dtype: int64

In [58]:
y_DDOS_HOIC_test[y_DDOS_HOIC_test>1.0] = 1.0

In [59]:
y_DDOS_HOIC_test = y_DDOS_HOIC_test.to_numpy()

In [60]:
DDOS_GOLDEN_EYE_test = pd.read_csv('/content/gdrive/Shareddrives/Anomaly/dataset/DDOS_GOLDEN_EYE.csv')

In [61]:
DDOS_GOLDEN_EYE_test.isna().sum()

Fwd Pkt Len Max      0
Fwd Pkt Len Min      0
Fwd Pkt Len Mean     0
Fwd Pkt Len Std      0
Bwd Pkt Len Max      0
Bwd Pkt Len Min      0
Bwd Pkt Len Mean     0
Bwd Pkt Len Std      0
Flow Byts/s          0
Flow Pkts/s          0
Bwd IAT Tot          0
Bwd IAT Mean         0
Bwd IAT Std          0
Bwd IAT Max          0
Bwd IAT Min          0
Fwd Pkts/s           0
Bwd Pkts/s           0
Pkt Len Min          0
Pkt Len Max          0
Pkt Len Mean         0
Pkt Len Std          0
Pkt Len Var          0
Pkt Size Avg         0
Fwd Seg Size Avg     0
Bwd Seg Size Avg     0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Fwd Seg Size Min     0
Idle Min             0
Label                0
dtype: int64

In [62]:
X_DDOS_GOLDEN_EYE_test = DDOS_GOLDEN_EYE_test.drop(labels = ['Label'], axis=1)

In [63]:
X_DDOS_GOLDEN_EYE_test = X_DDOS_GOLDEN_EYE_test.to_numpy()

In [64]:
y_DDOS_GOLDEN_EYE_test = DDOS_GOLDEN_EYE_test['Label']

In [65]:
y_DDOS_GOLDEN_EYE_test.value_counts()

1.0    41508
Name: Label, dtype: int64

In [66]:
y_DDOS_GOLDEN_EYE_test[y_DDOS_GOLDEN_EYE_test>1.0] = 1.0

In [67]:
y_DDOS_GOLDEN_EYE_test = y_DDOS_GOLDEN_EYE_test.to_numpy()

In [68]:
BENIGN_TEST_test = pd.read_csv('/content/gdrive/Shareddrives/Anomaly/dataset/BENIGN_TEST.csv')

In [69]:
BENIGN_TEST_test.isna().sum()

Fwd Pkt Len Max      0
Fwd Pkt Len Min      0
Fwd Pkt Len Mean     0
Fwd Pkt Len Std      0
Bwd Pkt Len Max      0
Bwd Pkt Len Min      0
Bwd Pkt Len Mean     0
Bwd Pkt Len Std      0
Flow Byts/s          0
Flow Pkts/s          0
Bwd IAT Tot          0
Bwd IAT Mean         0
Bwd IAT Std          0
Bwd IAT Max          0
Bwd IAT Min          0
Fwd Pkts/s           0
Bwd Pkts/s           0
Pkt Len Min          0
Pkt Len Max          0
Pkt Len Mean         0
Pkt Len Std          0
Pkt Len Var          0
Pkt Size Avg         0
Fwd Seg Size Avg     0
Bwd Seg Size Avg     0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Fwd Seg Size Min     0
Idle Min             0
Label                0
dtype: int64

In [70]:
X_BENIGN_TEST_test = BENIGN_TEST_test.drop(labels = ['Label'], axis=1)

In [71]:
X_BENIGN_TEST_test = X_BENIGN_TEST_test.to_numpy()

In [72]:
y_BENIGN_TEST_test = BENIGN_TEST_test['Label']

In [73]:
y_BENIGN_TEST_test.value_counts()

0.0    9946
Name: Label, dtype: int64

In [74]:
y_BENIGN_TEST_test[y_BENIGN_TEST_test>1.0] = 1.0

In [75]:
y_BENIGN_TEST_test = y_BENIGN_TEST_test.to_numpy()

In [76]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [77]:
class MiniDataset(Dataset):
  def __init__(self, X, y):
    super(MiniDataset, self).__init__()
    data, label = X, y
    self.data = torch.from_numpy(data)
    self.label = torch.from_numpy(label)
    self.n_classes = len(np.unique(label))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    image = self.data[index]
    label = self.label[index]
    return image, label

In [78]:
class MetaMiniDataset(MiniDataset):
  def __init__(self, X, y, n_batch=200, n_episode=4, n_way=2, n_shot=1000, n_query=8000):
    super(MetaMiniDataset, self).__init__(X=X, y=y)
    self.n_batch = n_batch
    self.n_episode = n_episode
    self.n_way = n_way
    self.n_shot = n_shot
    self.n_query = n_query

    self.catlocs = tuple()
    for cat in range(self.n_classes):
      self.catlocs += (np.argwhere(self.label == cat).reshape(-1),)

  def __len__(self):
    return self.n_batch * self.n_episode

  def __getitem__(self, index):
    shot, query = [], []
    cats = np.arange(self.n_classes)
    for c in cats:
      c_shot, c_query = [], []
      idx_list = np.random.choice(
        self.catlocs[c], self.n_shot + self.n_query, replace=False)
      shot_idx, query_idx = idx_list[:self.n_shot], idx_list[-self.n_query:]
      for idx in shot_idx:
        c_shot.append(self.data[idx])
      for idx in query_idx:
        c_query.append(self.data[idx])
      shot.append(torch.stack(c_shot))
      query.append(torch.stack(c_query))

    #print('T1', len(shot), shot[0].shape, len(query), query[0].shape)
    shot = torch.cat(shot, dim=0)             # [n_way * n_shot, C, H, W]
    query = torch.cat(query, dim=0)           # [n_way * n_query, C, H, W]
    #print('T2',len(shot), shot[0].shape, len(query), query[0].shape)
    cls = torch.arange(self.n_way)[:, None]
    shot_labels = cls.repeat(1, self.n_shot).flatten()    # [n_way * n_shot]
    query_labels = cls.repeat(1, self.n_query).flatten()  # [n_way * n_query]


    return shot, query, shot_labels, query_labels

In [79]:
train_set = MetaMiniDataset(X_train, y_train)

In [80]:
train_loader = DataLoader(train_set, shuffle=True)

In [81]:
epochs = 10
batch_size = 64
feats = 29
domains = 5
classes = 2
latent_dims = 7
learning_rate = 0.001

In [82]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [83]:
class AverageMeter(object):
  def __init__(self):
    self.reset()

  def reset(self):
    self.val = 0.
    self.avg = 0.
    self.sum = 0.
    self.count = 0.

  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

  def item(self):
    return self.avg

In [84]:
def get_child_dict(params, key=None):
  """
  Constructs parameter dictionary for a network module.

  Args:
    params (dict): a parent dictionary of named parameters.
    key (str, optional): a key that specifies the root of the child dictionary.

  Returns:
    child_dict (dict): a child dictionary of model parameters.
  """
  if params is None:
    return None
  if key is None or (isinstance(key, str) and key == ''):
    return params

  key_re = re.compile(r'^{0}\.(.+)'.format(re.escape(key)))
  if not any(filter(key_re.match, params.keys())):  # handles nn.DataParallel
    key_re = re.compile(r'^module\.{0}\.(.+)'.format(re.escape(key)))
  child_dict = OrderedDict(
    (key_re.sub(r'\1', k), value) for (k, value)
      in params.items() if key_re.match(k) is not None)
  return child_dict

In [85]:
class Module(nn.Module):
  def __init__(self):
    super(Module, self).__init__()
    self.efficient = False
    self.first_pass = True

  def go_efficient(self, mode=True):
    """ Switches on / off gradient checkpointing. """
    self.efficient = mode
    for m in self.children():
      if isinstance(m, Module):
        m.go_efficient(mode)

  def is_first_pass(self, mode=True):
    """ Tracks the progress of forward and backward pass when gradient
    checkpointing is enabled. """
    self.first_pass = mode
    for m in self.children():
      if isinstance(m, Module):
        m.is_first_pass(mode)

In [86]:
class Encoder(Module):
  def __init__(self):
    super(Encoder, self).__init__()
    self.linear1 = nn.Linear(29, 20)  # 29 * 20
    self.lin_bn1 = nn.BatchNorm1d(num_features=20)
    self.linear2 = nn.Linear(20, 10)  # 20 * 10
    self.lin_bn2 = nn.BatchNorm1d(num_features=10)
    self.linear3 = nn.Linear(10, 10)  # 10 * 10
    self.lin_bn3 = nn.BatchNorm1d(num_features=10)
    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(10, 7)  # 10 * 7

  def get_out_dim(self):
    return 7

  def forward(self, x):
    lin1 = self.relu(self.lin_bn1(self.linear1(x)))  # 29 * 20
    lin2 = self.relu(self.lin_bn2(self.linear2(lin1)))  # 20 * 10
    lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))  # 10 * 10

    fc1 = self.relu(self.fc1(lin3))  # 10 * 7

    return fc1

In [87]:
class MAML(Module):
  def __init__(self, encoder):
    super(MAML, self).__init__()
    self.encoder = encoder
    self.classifier = nn.Linear(7, 2)

  def reset_classifier(self):
    nn.init.zeros_(self.classifier.weight)
    nn.init.zeros_(self.classifier.bias)

  def _inner_forward(self, x, params, episode):
    """ Forward pass for the inner loop. """
    feat = self.encoder(x)
    logits = self.classifier(feat)
    return logits

  def _inner_iter(self, x, y, params, mom_buffer, episode, inner_args, detach):
    """
    Performs one inner-loop iteration of MAML including the forward and
    backward passes and the parameter update.

    Args:
      x (float tensor, [n_way * n_shot, C, H, W]): per-episode support set.
      y (int tensor, [n_way * n_shot]): per-episode support set labels.
      params (dict): the model parameters BEFORE the update.
      mom_buffer (dict): the momentum buffer BEFORE the update.
      episode (int): the current episode index.
      inner_args (dict): inner-loop optimization hyperparameters.
      detach (bool): if True, detachs the graph for the current iteration.

    Returns:
      updated_params (dict): the model parameters AFTER the update.
      mom_buffer (dict): the momentum buffer AFTER the update.
    """
    with torch.enable_grad():
      # forward pass
      logits = self._inner_forward(x, params, episode)
      loss = F.cross_entropy(logits, y)
      # backward pass
      grads = autograd.grad(loss, params.values(),
        create_graph=(not detach and not inner_args['first_order']),
        only_inputs=True, allow_unused=True)
      # parameter update
      updated_params = OrderedDict()
      for (name, param), grad in zip(params.items(), grads):
        if grad is None:
          updated_param = param
        else:
          if inner_args['weight_decay'] > 0:
            grad = grad + inner_args['weight_decay'] * param
          if inner_args['momentum'] > 0:
            grad = grad + inner_args['momentum'] * mom_buffer[name]
            mom_buffer[name] = grad
          if 'encoder' in name:
            lr = inner_args['encoder_lr']
          elif 'classifier' in name:
            lr = inner_args['classifier_lr']
          else:
            raise ValueError('invalid parameter name')
          updated_param = param - lr * grad
        if detach:
          updated_param = updated_param.detach().requires_grad_(True)
        updated_params[name] = updated_param

    return updated_params, mom_buffer

  def _adapt(self, x, y, params, episode, inner_args, meta_train):
    """
    Performs inner-loop adaptation in MAML.

    Args:
      x (float tensor, [n_way * n_shot, C, H, W]): per-episode support set.
        (T: transforms, C: channels, H: height, W: width)
      y (int tensor, [n_way * n_shot]): per-episode support set labels.
      params (dict): a dictionary of parameters at meta-initialization.
      episode (int): the current episode index.
      inner_args (dict): inner-loop optimization hyperparameters.
      meta_train (bool): if True, the model is in meta-training.

    Returns:
      params (dict): model paramters AFTER inner-loop adaptation.
    """
    # assert x.dim() == 4 and y.dim() == 1
    # assert x.size(0) == y.size(0)

    # Initializes a dictionary of momentum buffer for gradient descent in the
    # inner loop. It has the same set of keys as the parameter dictionary.
    mom_buffer = OrderedDict()
    if inner_args['momentum'] > 0:
      for name, param in params.items():
        mom_buffer[name] = torch.zeros_like(param)
    params_keys = tuple(params.keys())
    mom_buffer_keys = tuple(mom_buffer.keys())

    def _inner_iter_cp(episode, *state):
      """
      Performs one inner-loop iteration when checkpointing is enabled.
      The code is executed twice:
        - 1st time with torch.no_grad() for creating checkpoints.
        - 2nd time with torch.enable_grad() for computing gradients.
      """
      params = OrderedDict(zip(params_keys, state[:len(params_keys)]))
      mom_buffer = OrderedDict(
        zip(mom_buffer_keys, state[-len(mom_buffer_keys):]))

      detach = not torch.is_grad_enabled()  # detach graph in the first pass
      self.is_first_pass(detach)
      params, mom_buffer = self._inner_iter(
        x, y, params, mom_buffer, int(episode), inner_args, detach)
      state = tuple(t if t.requires_grad else t.clone().requires_grad_(True)
        for t in tuple(params.values()) + tuple(mom_buffer.values()))
      return state

    for step in range(inner_args['n_step']):
      if self.efficient:  # checkpointing
        state = tuple(params.values()) + tuple(mom_buffer.values())
        state = cp.checkpoint(_inner_iter_cp, torch.as_tensor(episode), *state)
        params = OrderedDict(zip(params_keys, state[:len(params_keys)]))
        mom_buffer = OrderedDict(
          zip(mom_buffer_keys, state[-len(mom_buffer_keys):]))
      else:
        params, mom_buffer = self._inner_iter(
          x, y, params, mom_buffer, episode, inner_args, not meta_train)

    return params

  def forward(self, x_shot, x_query, y_shot, inner_args, meta_train):
    """
    Args:
      x_shot (float tensor, [n_episode, n_way * n_shot, C, H, W]): support sets.
      x_query (float tensor, [n_episode, n_way * n_query, C, H, W]): query sets.
        (T: transforms, C: channels, H: height, W: width)
      y_shot (int tensor, [n_episode, n_way * n_shot]): support set labels.
      inner_args (dict, optional): inner-loop hyperparameters.
      meta_train (bool): if True, the model is in meta-training.

    Returns:
      logits (float tensor, [n_episode, n_way * n_shot, n_way]): predicted logits.
    """
    # assert self.encoder is not None
    # assert self.classifier is not None
    # assert x_shot.dim() == 5 and x_query.dim() == 5
    # assert x_shot.size(0) == x_query.size(0)

    # a dictionary of parameters that will be updated in the inner loop
    params = OrderedDict(self.named_parameters())
    for name in list(params.keys()):
      if not params[name].requires_grad or \
        any(s in name for s in inner_args['frozen'] + ['temp']):
        params.pop(name)

    logits = []
    for ep in range(x_shot.size(0)):
      # inner-loop training
      self.train()

      updated_params = self._adapt(
        x_shot[ep], y_shot[ep], params, ep, inner_args, meta_train)
      # inner-loop validation
      with torch.set_grad_enabled(meta_train):
        self.eval()
        logits_ep = self._inner_forward(x_query[ep], updated_params, ep)
      logits.append(logits_ep)

    self.train(meta_train)
    logits = torch.stack(logits)
    return logits

In [88]:
encoder = Encoder()
encoder = encoder.to(device)
model = MAML(encoder.float())
model = model.to(device)
model = model.float()

In [89]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3 )

In [90]:
aves_keys = ['tl', 'ta', 'vl', 'va']
trlog = dict()
for k in aves_keys:
  trlog[k] = []

In [91]:
def config_inner_args(inner_args):
  if inner_args is None:
    inner_args = dict()

  inner_args['reset_classifier'] = inner_args.get('reset_classifier') or False
  inner_args['n_step'] = inner_args.get('n_step') or 5
  inner_args['encoder_lr'] = inner_args.get('encoder_lr') or 0.01
  inner_args['classifier_lr'] = inner_args.get('classifier_lr') or 0.01
  inner_args['momentum'] = inner_args.get('momentum') or 0.
  inner_args['weight_decay'] = inner_args.get('weight_decay') or 0.
  inner_args['first_order'] = inner_args.get('first_order') or False
  inner_args['frozen'] = inner_args.get('frozen') or []

  return inner_args

In [92]:
inner_args = config_inner_args(None)

In [93]:
def compute_acc(pred, label, reduction='mean'):
  result = (pred == label).float()
  if reduction == 'none':
    return result.detach()
  elif reduction == 'mean':
    return result.mean().item()

In [94]:
def mean_confidence_interval(data, confidence=0.95):
  a = 1.0 * np.array(data)
  stderr = stats.sem(a)
  h = stderr * stats.t.ppf((1 + confidence) / 2., len(a) - 1)
  return h

In [95]:
for epoch in range(1, epochs):
    aves = {k: AverageMeter() for k in aves_keys}

    # meta-train
    model.train()
    np.random.seed(epoch)

    for data in train_loader:
      x_shot, x_query, y_shot, y_query = data
      x_shot, y_shot = x_shot.to(device), y_shot.to(device)
      x_query, y_query = x_query.to(device), y_query.to(device)

      #model.reset_classifier() #Check with commenting this out as well

      logits = model(x_shot.float(), x_query.float(), y_shot, inner_args, meta_train=True)
      logits = logits.flatten(0, 1)
      labels = y_query.flatten()

      pred = torch.argmax(logits, dim=-1)
      acc = compute_acc(pred.cpu(), labels.cpu())
      loss = F.cross_entropy(logits, labels)
      aves['tl'].update(loss.item(), 1)
      aves['ta'].update(acc, 1)

      optimizer.zero_grad()
      loss.backward()
      for param in optimizer.param_groups[0]['params']:
        nn.utils.clip_grad_value_(param, 10)
      optimizer.step()

    for k, avg in aves.items():
      aves[k] = avg.item()
      trlog[k].append(aves[k])
    log_str = 'epoch {}, meta-train {:.4f}|{:.4f}'.format(
      str(epoch), aves['tl'], aves['ta'])
    print(log_str)

epoch 1, meta-train 0.2254|0.9205
epoch 2, meta-train 0.0599|0.9843
epoch 3, meta-train 0.0485|0.9884
epoch 4, meta-train 0.0412|0.9900
epoch 5, meta-train 0.0376|0.9907
epoch 6, meta-train 0.0356|0.9910
epoch 7, meta-train 0.0350|0.9913
epoch 8, meta-train 0.0330|0.9922
epoch 9, meta-train 0.0290|0.9927


In [96]:
def test_model(X, y, n_shot=100, n_query=1000):
  test_set = MetaMiniDataset(X, y, n_shot=n_shot, n_query=n_query)
  test_loader = DataLoader(test_set, shuffle=True)
  model.eval()
  aves_va = AverageMeter()
  va_lst = []
  labels_lst = []
  predict_lst = []
  for epoch in range(1, 2):
    for data in test_loader:
      x_shot, x_query, y_shot, y_query = data
      x_shot, y_shot = x_shot.to(device), y_shot.to(device)
      x_query, y_query = x_query.to(device), y_query.to(device)

      # if inner_args['reset_classifier']:
      #   if config.get('_parallel'):
      #     model.module.reset_classifier()
      #   else:
      #     model.reset_classifier()

      logits = model(x_shot.float(), x_query.float(), y_shot, inner_args, meta_train=False)
      # logits = logits.view(-1, 2)
      # labels = y_query.view(-1)
      logits = logits.flatten(0, 1)
      labels = y_query.flatten()

      pred = torch.argmax(logits, dim=1)
      predict_lst.extend(pred.cpu().detach().numpy())
      labels_lst.extend(labels.cpu().detach().numpy())
      acc = compute_acc(pred, labels)

      aves_va.update(acc, 1)
      va_lst.append(acc)

    print('test epoch {}: acc={:.2f} +- {:.2f} (%)'.format(
      epoch, aves_va.item() * 100,
      mean_confidence_interval(va_lst) * 100))
  return labels_lst, predict_lst

In [97]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

def test(X_test, y_test, n_shot=100, n_query=1000):
  labels_lst, predict_lst = test_model(X_test, y_test, n_shot, n_query)
  prec = precision_score(labels_lst, predict_lst)
  rec = recall_score(labels_lst, predict_lst)
  f1 = f1_score(labels_lst, predict_lst)
  roc_auc = roc_auc_score(labels_lst, predict_lst)
  print(prec, rec, f1, roc_auc)

In [98]:
test(np.concatenate((X_BOT_test[:10000], X_BENIGN_TEST_test[:10000])), np.concatenate((y_BOT_test[:10000], y_BENIGN_TEST_test[:10000])))

test epoch 1: acc=38.12 +- 0.74 (%)
0.3258090650478034 0.22218875 0.2642021807890794 0.38120875


In [99]:
test(np.concatenate((X_RARE_test[:566], X_BENIGN_TEST_test[:566])), np.concatenate((y_RARE_test[:566], y_BENIGN_TEST_test[:566])), 50, 500)

test epoch 1: acc=47.37 +- 0.24 (%)
0.4735289817442992 0.46962 0.47156639027586367 0.47374750000000004


In [100]:
test(np.concatenate((X_SlowHTTPTest_test[:10000], X_BENIGN_TEST_test[:10000])), np.concatenate((y_SlowHTTPTest_test[:10000], y_BENIGN_TEST_test[:10000])))

test epoch 1: acc=90.09 +- 0.12 (%)
0.8349248220018087 0.99939875 0.9097880397862077 0.9009025


In [101]:
test(np.concatenate((X_OOD_INFIL_test[:10000], X_BENIGN_TEST_test[:10000])), np.concatenate((y_OOD_INFIL_test[:10000], y_BENIGN_TEST_test[:10000])))

test epoch 1: acc=70.31 +- 0.24 (%)
0.7668210484824596 0.5836275 0.6627988189199931 0.7030775


In [102]:
test(np.concatenate((X_DDOS_SOLARIS_test[:10000], X_BENIGN_TEST_test[:10000])), np.concatenate((y_DDOS_SOLARIS_test[:10000], y_BENIGN_TEST_test[:10000])))

test epoch 1: acc=52.59 +- 0.24 (%)
0.526624133803866 0.5125975 0.5195161564130305 0.525915


In [103]:
test(np.concatenate((X_DDOS_HOIC_test[:10000], X_BENIGN_TEST_test[:10000])), np.concatenate((y_DDOS_HOIC_test[:10000], y_BENIGN_TEST_test[:10000])))

test epoch 1: acc=89.46 +- 0.43 (%)
0.8411609704007453 0.97283 0.9022168689783251 0.8945637500000001


In [104]:
test(np.concatenate((X_DDOS_GOLDEN_EYE_test[:10000], X_BENIGN_TEST_test[:10000])), np.concatenate((y_DDOS_GOLDEN_EYE_test[:10000], y_BENIGN_TEST_test[:10000])))

test epoch 1: acc=24.12 +- 0.17 (%)
0.08675008033338855 0.05433125 0.06681593221407231 0.24118312500000003
